In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

In [2]:
images=glob.glob('./camera_cal/calibration*.jpg')
objpoints=[]
imgpoints=[]
ava_images=[]
objp=np.zeros((6*9,3),np.float32)
#print(objp[:,:2])
objp[:,:2]=np.mgrid[0:9,0:6].T.reshape(-1,2) #x,y的坐标
#print(objp[:,:2])

for frame in images:
    img=mpimg.imread(frame)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,corners=cv2.findChessboardCorners(gray,(9,6),None)
    if ret==True:
        #print(corners)
        ava_images.append(frame)
        imgpoints.append(corners)
        objpoints.append(objp)



In [3]:
# 1.Camera calibration
# 2.Distortion correction
def cal_undistort(img,objpoints,imgpoints):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,mtx,dist,rvecs,tvecs=cv2.calibrateCamera(objpoints,imgpoints,gray.shape[::-1],None,None)
    dst=cv2.undistort(img,mtx,dist,None,mtx)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    undist = np.copy(dst)  # Delete this line
    return undist

In [4]:
# 3.Color/gradient threshold
def pre_deal(img,s_thresh=(170,255),sx_thresh=(20,100)):
    img=np.copy(img)
    hls=cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    s_channel=hls[:,:,2]
    gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    #sobelx
    sobelx=cv2.Sobel(gray,cv2.CV_64F,1,0)
    abs_sobelx=np.absolute(sobelx)
    scaled_sobel=np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sxbinary=np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel>=sx_thresh[0])&(scaled_sobel<=sx_thresh[1])]=1
    
    s_binary=np.zeros_like(s_channel)
    s_binary[(s_channel>=s_thresh[0])&(s_channel<=s_thresh[1])]=1
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary

In [6]:
# 4.Perspective transform
def perspective_transform(img):
    img_size=(img.shape[1],img.shape[0])
    src=np.float32([[595,450],[685,450],[1050,680],[280,680]])
    dst=np.float32([[300,50],[950,50],[950,720],[300,720]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv=cv2.getPerspectiveTransform(dst, src)
    warped=cv2.warpPerspective(img,M,img_size)
    return warped,Minv

In [9]:
# 5.Detect lane lines
def find_lines(binary_warped):

    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
#         cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
#         (0,255,0), 2) 
#         cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
#         (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        #因为np.nozero()函数的作用是找到矩阵中的非零元素的横纵坐标（索引）
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
       # print(good_right_inds )
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])                        
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])              
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
#     combined_left_window=np.array([np.transpose(np.vstack([left_fitx, ploty]))])
#     combined_right_window=np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
#     combined_window=np.hstack((combined_left_window, combined_right_window))
                               
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    
#     cv2.fillPoly(window_img,np.int_([combined_window]),(0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    return left_fitx,right_fitx


In [7]:
#Lanes_find
def main_fuction(img,objpoints=objpoints,imgpoints=imgpoints,s_thresh=(170,255),sx_thresh=(20,100)):
    # 1.Camera calibration
    # 2.Distortion correction
    undist=cal_undistort(img,objpoints,imgpoints)
    # 3.Color/gradient threshold
    combined_binary=pre_deal(undist,s_thresh=(170,255),sx_thresh=(20,100))
    # 4.Perspective transform
    per_tran,Minv=perspective_transform(combined_binary)
    # 5.Detect lane lines
    left_fitx,right_fitx=find_lines(per_tran)
    warp_zero = np.zeros_like(combined_binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    # 6.Determine the lane curvature
    y_eval = np.max(ploty)
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    

    # Fit new polynomials to x,y in world space
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    leftx_int =left_fitx[-1]
    rightx_int = right_fitx[-1]
    center = 640 - ((rightx_int+leftx_int)/2)
    if center>=0:
#         plt.text(300, 100, 'Vehicle is {:.2f}m right of center'.format(center*3.7/700),
#                  style='italic', color='white', fontsize=10)
        cv2.putText(result, 'Vehicle is {:.2f}m right of center'.format(center*3.7/700), (300,100),
                 fontFace = 16, fontScale = 1, color=(255,255,255), thickness = 2)
    else:
#         plt.text(300, 100, 'Vehicle is {:.2f}m left of center'.format(abs(center)*3.7/700),
#                  style='italic', color='white', fontsize=10)
        cv2.putText(result, 'Vehicle is {:.2f}m left of center'.format(abs(center*3.7/700)), (300,100),
                 fontFace = 16, fontScale = 1, color=(255,255,255), thickness = 2)


#     plt.text(300, 175, 'Radius of curvature is {}m'.format(int((left_curverad + right_curverad)/2)),
#              style='italic', color='white', fontsize=10)
    cv2.putText(result, 'Radius of curvature is {}m'.format(int((left_curverad + right_curverad)/2)), (300,175),
                 fontFace = 16, fontScale = 1, color=(255,255,255), thickness = 2)
    return result


In [10]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

output = 'test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,2)
clip1 = VideoFileClip("test_videos/project_video.mp4")
white_clip = clip1.fl_image(main_fuction) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video test_videos_output/project_video.mp4
[MoviePy] Writing video test_videos_output/project_video.mp4


 96%|█████████▌| 25/26 [00:21<00:00,  1.16it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/project_video.mp4 

CPU times: user 39.7 s, sys: 435 ms, total: 40.1 s
Wall time: 21.9 s


In [11]:
HTML("""
<video width="640" height="360" controls>
  <source src="{0}">
</video>
""".format(output))